<a href="https://colab.research.google.com/github/molnaredom/gepitan/blob/main/gepitan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Adatok előkészítése

##--Adat beolvasás:

In [ ]:
import pandas as pd
df = pd.read_csv('https://datahub.io/machine-learning/credit-approval/r/credit-approval.csv', 
                 sep=',',  # \t ha tab , " " ha szokoz
                 header=0, # 0 --> a 0. sor a header, None --> nincs header
                 #  nrows=86 # ennyi sort veszünk figyelembe
                 # skiprows = 3 az eslő ennyi sort hagyjuk figyelmen kívül
                 # names = colnames # Ha külön van a header itt külön is hozzá tudjuk adni
                 #na_values="?"
                 )
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,class
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


##--Adathalmaz szétválasztása

(1)Adathalmaz szétválasztása cél és jellemző változókra




In [ ]:
# célváltozó megadása 
classlabel = df["class"] #névvel
classlabel = df.iloc[:,-1] #számmal (összes sor) (utolsó oszlop)

#jellemzőoszlopok kinyerése
features = df.iloc[:,:-1] # az utolsó oszlop kivételével minden oszlop 



```
# Ez kódként van formázva
```

(2)Adathalmaz szétválasztása cél és jellemző változókra

In [ ]:
from sklearn.model_selection import train_test_split
train_features, valid_features, train_classlabels, valid_classlabels= train_test_split(features,
                                                                                       classlabels,
                                                                                       test_size=0.25,
                                                                                       random_state=42)

SyntaxError: ignored


(3)Adathalmaz szétválasztása cél és jellemző változókra

In [ ]:
testFeatures = d[d.GameID > 7000].loc[:,"Age":]
trainFeatures = d[d.GameID < 7001].loc[:,"Age":]
trainLabels = d[d.GameID < 7001].LeagueIndex
testLabels =  d[d.GameID > 7000].LeagueIndex

##--Átalakítások, szűrések

Nan értékek eldobása✈

In [ ]:
df = df.dropna()

Diszkrét jellemzőkké alakítás

In [ ]:
from sklearn import preprocessing
ohe = preprocessing.OneHotEncoder()
cat_features = ohe.fit_transform(features.select_dtypes(include=['object'])) # kategórikus jellemzők ritka mátrixként
import scipy.sparse
num_features = scipy.sparse.csr_matrix(features.select_dtypes(exclude=['object'])) # numerikus jellemzők ritka mátrixként
ohe_features = scipy.sparse.hstack((cat_features, num_features)) # ritka mátrixok horizontális konkatenálása 
ohe_features

Szózsákokba osztás --> Ha szöveget kell kezelni

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( # szózsák
    min_df=10) # ????? -> ez hamis --azokat a szavakat jegyzi meg amiből 9 nél több volt

vectorizer.fit(train_data.text)# minden előforduló szót összegyűjt

features = vectorizer.transform(train_data.text) #minden szót megszámol hányszor fordul elő 
                                                 #és hozzárendeli a szóhoz a számértéket
                                                 

Szöveg feldolgozás

#Model típusok



---



##-- Osztályozás



Osztályozási Döntési fa  --> Ha kevés jellemző oszlop van

In [ ]:
from sklearn import tree
# döntési fa osztályozó  --- kevés számú diszkrét jellemző esetén.
dt = tree.DecisionTreeClassifier(
     min_samples_leaf=20,
     max_depth=2
     ) 

Lineáris gépek  --> Ha sok jellemző oszlop van (pl szózsák)

In [ ]:
from sklearn.linear_model import SGDClassifier
cls = SGDClassifier()

model = cls.fit(features, train_data.label)

k legközelebbi szomszéd knn --> ha figyelembe akarjuk venni hogy egymás mellett vannak ez az adatok (pixelek)(pl képek)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=15) # n_neighbors a k érték (szomszédok száma)

##--Regresszió

Lineáris regressziós modell - lineáris gép szintén - sok oslop eseten hasznos

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression() # lineáris gép regresszióra
reg.fit(trainFeatures, trainLabels)
prediction = reg.predict(testFeatures)

Regressziós(folytonos értékek) Döntési fa --> ha kevés oszlop van

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

dt = KNeighborsRegressor(n_neighbors=9) # döntési fa regresszióra
dt.fit(trainFeatures, trainLabels) # tanítás a tanító adatbázison
prediction = dt.predict(testFeatures) # predikció a teszt adatbázison

Logistic regression - egyenesekkel dönti el hova esnek az egyedek

In [ ]:
lin = LogisticRegression(random_state=0, max_iter=10000, class_weight="balanced", C=10**-3)
lin.fit(train_features, train_classlabels)  
valid_prediction = lin.predict(valid_features)
print(classification_report(valid_classlabels, valid_prediction))
print("Legjobb logisztikus regresszió a validációs halmazon:", f1_score(valid_classlabels, valid_prediction, pos_label=1))

# Model meta paraméter hangolás

##--Döntési fa hangolása - mélység beállítása (melyik mélység az ideális?)

In [ ]:
valid_f1=[]
train_f1=[]
for d in range(1,40):
  dt = tree.DecisionTreeClassifier(max_depth=d) # döntési fa mélysége
  dt.fit(train_features, train_classlabels)  # tanítunk a tanító adatbázison
  valid_prediction = dt.predict(valid_features)
  valid_f1.append(f1_score(valid_classlabels, valid_prediction, pos_label=1)) # kiértékelés a validációs halmazon
  train_prediction = dt.predict(train_features)
  train_f1.append(f1_score(train_classlabels, train_prediction, pos_label=1)) # a tanító adatbázison is kiértékeljük a túltanulási vizsgálatokhoz

#kirazolás
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.plot(valid_f1, c="green")
plt.plot(train_f1, c="red")

## --Lineáris gép meta-paraméter hangolása


Cél a  legjobb regularizációs együttható megtalálása ( l betűvel jelezzük)

In [ ]:
from sklearn.linear_model import LogisticRegression
valid_f1=[]
train_f1=[]
for l in range(-7,5):
  lin = LogisticRegression(random_state=0, 
                           max_iter=10000, # nehéz feladat a lineáris gépnek, engedjük a defaultnál tovább tanulni
                           class_weight="balanced", # tanulásnál átsúlyozzuk a ritka osztálycímkéket, ez sokat segít lineáris gépeknél
                           C=10**l) # regularizációs együttható 
  lin.fit(train_features, train_classlabels)  
  valid_prediction = lin.predict(valid_features)
  valid_f1.append(f1_score(valid_classlabels, valid_prediction, pos_label=1))
  train_prediction = lin.predict(train_features)
  train_f1.append(f1_score(train_classlabels, train_prediction, pos_label=1))

#kirajzolás
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.plot(valid_f1, c="green")
plt.plot(train_f1, c="red")

# aza jó ahol a zöld a legmagasabb mert aza  validációs halamaz

## --kNN osztályozó meta-paraméter hangolása

Azt keressük hány szomszéddal nézva a legjobb az eredmény a validácis halmazon

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
valid_f1=[]
train_f1=[]
for k in range(1,20,2): # páratlan k értékek
  knn = KNeighborsClassifier(n_neighbors=k) # kNN különböző k értékekkel
  knn.fit(train_features, train_classlabels)  
  valid_prediction = knn.predict(valid_features)
  valid_f1.append(f1_score(valid_classlabels, valid_prediction, pos_label=1))
  train_prediction = knn.predict(train_features)
  train_f1.append(f1_score(train_classlabels, train_prediction, pos_label=1))

In [ ]:
### kiiratas
plt.figure(figsize=(10,10))
plt.plot(valid_f1, c="green")
plt.plot(train_f1, c="red")

## --*? HA az összes előfordulási lehetőségre meg akarjuk nezni

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

parameters = {'kernel':['linear', 'rbf'], 'C':[100, 10, 1]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters,
                   cv = 3,
                   scoring =  make_scorer(f1_score, pos_label=1),
                   verbose=3)
clf.fit(features, classlabels)

In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
clf.best_estimator_

#Predikciók

##Főmodell kiértékelés

 precision    recall  f1-score   support

In [ ]:
prediction = dt.predict(valid_features)
from sklearn.metrics import classification_report
print(classification_report(valid_classlabels, prediction))


Accuracy

In [ ]:
accuracy_score(y_true=test_data.label, y_pred=prediction)

#Kiértékelés

##--Visszaellenőrzés-jó e amit csináltunk BASELINE-al

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent") #vagy  mean # tanító adatbázis leggyakoribb osztálya lesz mindig a predikció
dummy_clf.fit(features, train_data.label) # ugyanazon a tanító adatbázison "tanítjuk"
baseline_prediction = dummy_clf.predict(test_features) # predikció a kiértékelő adatbázison
accuracy_score(baseline_prediction, test_data.label)

MSE hibaellenőrzés

In [ ]:
from sklearn.metrics import mean_squared_error # MSE (RMSE végső gyökvonás nélkül)
mean_squared_error(prediction, testLabels)

from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean') # tanító adatbázis címkéinek átlaga lesz mindig a predikció
dummy.fit(trainFeatures, trainLabels)
mean_squared_error(dummy.predict(testFeatures), testLabels)

##--Ábrák, kirajzolások

Sima hisztogramm

In [ ]:
classlabels.hist()

Legértékesebb, legerősebb jellemző kinyerése - osztályozás esetén

In [ ]:
sorted(zip(model.coef_[0], vectorizer.get_feature_names()),reverse=True)[:15]

legerősebb jellemző regressziós esetén

In [ ]:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(trainFeatures, trainLabels) # tanítás a tanító adatbázison
root = dt.tree_.feature[0] # gyökér csúcsban lévő jellemző indexe
trainFeatures.columns[root] # legfontosabb jellemző DT szerint

Döntési fa kirajzolás

In [ ]:
!apt-get -qq install -y graphviz && pip install -q pydot #Thank you https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d
!pip install graphviz
import pydot
import graphviz
graphviz.Source(tree.export_graphviz(dt, out_file=None))

Prediktálás precision recall..

Regresszió vizualizalas:

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
plt.plot(range(0,len(trainLabels)), trainLabels, c='grey', alpha=0.5)
testrange = range(len(trainLabels)-1,len(trainLabels)+len(testLabels)-1)
plt.plot(testrange, testLabels, c='c', marker='o')
plt.plot(testrange, prediction, c='r', marker='o')
ax.legend(['múlt', 'jövőbeli tényleges', 'predikció'])
plt.show()

Tévesztési mátrix szines

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(y_true=test_data.label, y_pred=prediction)
ConfusionMatrixDisplay(cm, display_labels=model.classes_).plot(values_format='.3g')